# checking on foil data part 1.3

In [ ]:
# %load imports.py
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import pandas
import scipy as sp
import scipy.ndimage as snd
import scipy.signal as ss
import mkl_fft
import fast_histogram
import sys, os, datetime, shutil, glob, itertools, collections
import h5py
#from tqdm.notebook import trange, tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib as mpl
import tqdm.notebook as tqdm
import skimage.morphology as skm
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
plt.rcParams['figure.figsize']=(15,7)
filenames=glob.glob('/UserData/gorkhover/TAIS2020/foils/p1/new2/*.h5')
firstrun=[h5py.File(f,'r')['meta/runs'][0] for f in filenames]
filenames=[x for _,x in sorted(zip(firstrun,filenames))]

In [ ]:
#create result dirs
from pathlib import Path

for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    print(name)
    Path(f"/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}").mkdir(parents=True, exist_ok=True)

In [ ]:

for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    f=h5py.File(filename,'r')
    firstrun=f['meta/runs'][0].decode('ascii')
    lastrun=f['meta/runs'][-1].decode('ascii')


    md=np.array(f['detectors/dual/mean'][50:-50,50:-150])
    mo=np.array(f['detectors/octal/mean'][200:-200,250:-250])
    vd=np.square(np.array(f['detectors/dual/std'][50:-50,50:-150]))
    vo=np.square(np.array(f['detectors/octal/std'][200:-200,250:-250]))
    fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(20,10))
    qd=vd/md
    qo=vo/mo
    im1=ax1.matshow(qd,vmin=np.nanpercentile(qd,1),vmax=np.nanpercentile(qd,99))
    ax1.set_title(f'var/mean dual')
    plt.colorbar(im1,ax=ax1)
    im2=ax2.matshow(qo,vmin=np.nanpercentile(qo[200:-200,200:-200],1),vmax=np.nanpercentile(qo[200:-200,200:-200],99))
    ax2.set_title(f'var/mean octal')
    plt.colorbar(im2,ax=ax2)
    fig.suptitle(f'{name}/{firstrun}-{lastrun}')
    fig.tight_layout(rect=[0.05, 0.1, .95, 0.9])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'varmeanquotient.png')
    plt.close()

## used Backgrounds

In [ ]:
for filename in filenames:#[filenames[0],filenames[-1]]:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    f=h5py.File(filename,'r')
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'

    firstrun=f['meta/runs'][0].decode('ascii')
    lastrun=f['meta/runs'][-1].decode('ascii')

    bgd=np.array(f['detectors/dual/bg'][50:-50,50:-150])
    bgo=np.array(f['detectors/octal/bg'][200:-200,250:-250])
    fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(20,10))
    im1=ax1.matshow(bgd,vmin=np.percentile(bgd[bgd!=0],5),vmax=np.percentile(bgd[bgd!=0],95))
    ax1.set_title(f'background used on dual')
    plt.colorbar(im1,ax=ax1)
    im2=ax2.matshow(bgo,vmin=np.percentile(bgo[bgo!=0],5),vmax=np.percentile(bgo[bgo!=0],95))
    ax2.set_title(f'background used on octal')
    plt.colorbar(im2,ax=ax2)
    fig.suptitle(f'{name}/{firstrun}-{lastrun}')
    fig.tight_layout(rect=[0.05, 0.1, .95, 0.9])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'bg.png')
    plt.close()

## Mean images ##

In [ ]:


for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    f=h5py.File(filename,'r')
    firstrun=f['meta/runs'][0].decode('ascii')
    lastrun=f['meta/runs'][-1].decode('ascii')
    md=np.array(f['detectors/dual/mean'][50:-50,50:-150])
    mo=np.array(f['detectors/octal/mean'][200:-200,250:-250])
    sd=np.array(f['detectors/dual/std'][50:-50,50:-150])
    so=np.array(f['detectors/octal/std'][200:-200,250:-250])
    fig,((ax1,ax2),(ax3,ax4))=plt.subplots(ncols=2,nrows=2,figsize=(20,20))
    im1=ax1.matshow(md,vmin=np.mean(md[md!=0])*0.8,vmax=np.mean(md[md!=0])*1.2)
    ax1.set_title(f'mean dual')
    plt.colorbar(im1,ax=ax1)
    im2=ax2.matshow(mo,vmin=np.mean(mo[mo!=0])*0.8,vmax=np.mean(mo[mo!=0])*1.2)#,vmin=np.percentile(mo,10),vmax=np.percentile(mo,99))
    ax2.set_title(f'mean octal')
    plt.colorbar(im2,ax=ax2)
    im3=ax3.matshow(sd,vmin=np.mean(sd[md!=0])*0.8,vmax=np.mean(sd[md!=0])*1.2)
    ax3.set_title(f'std dual')
    plt.colorbar(im3,ax=ax3)
    im4=ax4.matshow(so,vmin=np.mean(so[mo!=0])*0.8,vmax=np.mean(so[mo!=0])*1.2)#,vmin=np.percentile(mo,10),vmax=np.percentile(mo,99))
    ax4.set_title(f'std octal')
    plt.colorbar(im4,ax=ax4)
    fig.suptitle(f'{name}/{firstrun}-{lastrun}')
    fig.tight_layout(rect=[0.05, 0.05, .95, 0.95])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'meanstd_ev.png')
    plt.close()

## histograms

In [ ]:
for filename in tqdm.tqdm(filenames):
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    f=h5py.File(filename,'r')
    shots=np.sum(f['meta/filtering/shot_ok'],axis=1)
    labels=[s.decode('ascii') for s in f['meta/runs'][shots>0] ]
    shots=shots[shots>0]
    hist_d=(np.array(f['detectors/dual/hist/run'])/shots[:,None]).T
    hist_o=(np.array(f['detectors/octal/hist/run'])/shots[:,None]).T
    bins=f['detectors/dual/hist/bincenters']
    kalpha=int(np.array(f['meta/Kalpha']))
    fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(20,10))
    ax1.semilogy(bins,hist_d)
    ax1.set_xlim(3000,5.5*kalpha)
    ax1.legend(labels,loc='best',ncol=2)
    ax1.set_title(f'dual (±1500ev)')
    for n in range(5): ax1.axvline(kalpha*n,c='green')
    for n in range(5): ax1.axvspan(kalpha*n-1500,kalpha*n+1500,facecolor='green',alpha=0.4)
    ax1.axvline(np.nanmean(f['photonEnergy']),c='red')
    ax1.axvspan(np.nanmean(f['photonEnergy'])-1000,np.nanmean(f['photonEnergy'])+1000,facecolor='red',alpha=0.4)

    ax2.semilogy(bins,hist_o)
    ax2.set_xlim(-1000,5.5*kalpha)
    ax2.set_title(f'octal (±1500ev)')
    for n in range(6): ax2.axvline(kalpha*n,c='green')
    for n in range(6): ax2.axvspan(kalpha*n-1500,kalpha*n+1500,facecolor='green',alpha=0.4)
    ax2.axvspan(np.nanmean(f['photonEnergy'])-1000,np.nanmean(f['photonEnergy'])+1000,facecolor='red',alpha=0.4)

    ax2.axvline(np.nanmean(f['photonEnergy']),c='red')
    ax2.legend(labels,loc='best',ncol=2)
    fig.suptitle(f'{name}')
    fig.tight_layout(rect=[0.05, 0.05, .95, 0.95])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'spectrum.png')
    plt.close()

## scanning

In [ ]:
for filename in tqdm.tqdm(filenames):
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    
    f=h5py.File(filename,'r')

    samplez=np.array(f['motors/sampleZ'])+np.array(f['motors/profZ'])
    samplex=np.array(f['motors/sampleX'])

    intensity=np.array(f['detectors/dual/intensity'])
    fig,(ax1,ax2,ax3)=plt.subplots(nrows=1,ncols=3,figsize=(27,8))
    im1=ax1.scatter(samplex,samplez,c=intensity,s=2)#,vmin=np.percentile(p6[beamok]/beam[beamok],5),vmax=np.percentile(p6[beamok]/beam[beamok],95))
    ax1.set_title('intensity dual')
    ax1.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax1.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im1,ax=ax1)
    ax2.plot(np.array(f['motors/profZ']),label='profZ')
    
    ax2.plot(np.array(f['motors/sampleZ']),label='sampleZ')
    ax2.legend(loc='upper right')
    ax3.plot(np.array(f['motors/sampleX']),label='sampleX')
    ax3.legend(loc='upper right')
    
    fig.suptitle(f'{name}')
    fig.tight_layout(rect=[0.05, 0.05, .95, 0.95])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'scanning.png')
    plt.close()

## strange peak shape in 'Fe500nm_defocus+400'

In [ ]:
name='Fe500nm_defocus+400'
filename=f'/UserData/gorkhover/TAIS2020/foils/p1/new2/{name}_p1.h5'

    
f=h5py.File(filename,'r')
shots=np.sum(f['meta/filtering/shot_ok'],axis=1)
labels=[s.decode('ascii') for s in f['meta/runs'][shots>0] ]
shots=shots[shots>0]
hist_d=(np.array(f['detectors/dual/hist/run'])/shots[:,None]).T
hist_o=(np.array(f['detectors/octal/hist/run'])/shots[:,None]).T
bins=f['detectors/dual/hist/bincenters']
kalpha=int(np.array(f['meta/Kalpha']))
startrun=22
endrun=27

startshot=np.cumsum(shots)[startrun]
endshot=np.cumsum(shots)[endrun]

fig,(ax1,ax2)=plt.subplots(ncols=2)
ax1.semilogy(bins,hist_d[:,start:stop])
ax1.set_xlim(4000,10000)
ax1.legend(labels[start:stop],loc='best',ncol=2)
ax1.set_title(f'{name} dual')
for n in range(4): ax1.axvline(kalpha*n,c='green')
ax1.axvline(np.nanmean(f['photonEnergy']),c='red')
plt.show()
#         ax2.semilogy(bins,hist_o)
#         ax2.set_xlim(-1000,30000)
#         ax2.set_title(f'{name} octal')
#         for n in range(4): ax2.axvline(kalpha*n,c='green')
#         ax2.axvline(np.nanmean(f['photonEnergy']),c='red')
#         ax2.legend(labels,loc='best',ncol=2)
#         plt.show()

In [ ]:
startrun=22#22
endrun=23#27

startshot=np.cumsum(shots)[startrun]
endshot=np.cumsum(shots)[endrun]
hists=f['detectors/dual/hist/shot'][startshot:endshot]
print(len(hists))
step=10
for i in range(len(hists)//step):
    starth=i*step
    stoph=(i+1)*step
    hist=np.sum(hists[starth:stoph,:],axis=0)
    plt.semilogy(bins,hist)
    plt.xlim(-1000,10000)

In [ ]:
name='Fe500nm_defocus+400'
filename=f'/UserData/gorkhover/TAIS2020/foils/p1/new2/{name}_p1.h5'  
f=h5py.File(filename,'r')
samplex=f['motors/sampleX']
samplez=f['motors/sampleZ']
histd=np.array(f['detectors/dual/hist/shot'])
p8d=np.sum(histd[:,1000:1100],axis=1)
plt.rcParams['figure.figsize']=(20,15)
plt.scatter(samplex,samplez,c=p8d,s=4,)
plt.xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
plt.ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
plt.colorbar()

## peaks vs sample position

In [ ]:
plt.rcParams['figure.figsize']=(20,10)
for filename in tqdm.tqdm(filenames):
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    
    f=h5py.File(filename,'r')


    samplex=np.array(f['motors/sampleX'])
    samplez=np.array(f['motors/sampleZ'])+np.array(f['motors/profZ'])
    intensity=np.array(f['detectors/dual/intensity'])
    beam=np.array(f['pulse_energy_hutch_joule'])
    histd=np.array(f['detectors/dual/hist/shot'])
    p8d=np.sum(histd[:,1000:1100],axis=1)
    p6d=np.sum(histd[:,750:850],axis=1)
    p11d=np.sum(histd[:,1250:1350],axis=1)
    
    histo=np.array(f['detectors/octal/hist/shot'])
    p8o=np.sum(histo[:,1000:1100],axis=1)
    p6o=np.sum(histo[:,750:850],axis=1)
    p11o=np.sum(histo[:,1250:1350],axis=1)
    
    norm=np.mean(beam)/beam
    
    fig,((ax1,ax3),(ax2,ax4),(ax5,ax6))=plt.subplots(nrows=3,ncols=2,figsize=(20,10))
    im1=ax1.scatter(samplex,samplez,c=p6d*norm,s=2,vmin=np.mean(p6d)*0.5)
                    #,vmin=np.percentile(p6[beamok]/beam[beamok],5),vmax=np.percentile(p6[beamok]/beam[beamok],95))
    ax1.set_title('iron dual')
    ax1.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax1.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im1,ax=ax1)


    im2=ax2.scatter(samplex,samplez,c=p8d*norm,s=2,vmin=np.mean(p8d)*0.5)#,vmax=np.mean(p8d)*1.25)
    ax2.set_title('copper dual')
    ax2.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax2.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im2,ax=ax2)


    im3=ax3.scatter(samplex,samplez,c=p6o*norm,s=2,vmin=np.mean(p6o)*0.5)#,vmax=np.mean(p6o)*1.25)
    ax3.set_title('iron octal')
    ax3.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax3.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im3,ax=ax3)


    im4=ax4.scatter(samplex,samplez,c=p8o*norm,s=2,vmin=np.mean(p8o)*0.5)#,vmax=np.mean(p8o)*1.25)
    ax4.set_title('copper octal')
    ax4.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax4.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im4,ax=ax4)


    im5=ax5.scatter(samplex,samplez,c=p11d*norm,s=2,vmin=np.mean(p11d)*0.5)#,vmax=np.mean(p11d)*1.25)
    ax5.set_title('scatter dual')
    ax5.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax5.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im5,ax=ax5)

    im6=ax6.scatter(samplex,samplez,c=p11o*norm,s=2,vmin=np.mean(p11o)*0.5)#,vmax=np.mean(p11o)*1.25)
    ax6.set_title('scatter octal')
    ax6.set_xlim(np.min(samplex)-0.0005,np.max(samplex)+0.00051)
    ax6.set_ylim(np.min(samplez)-0.00051,np.max(samplez)+0.00051)
    plt.colorbar(im6,ax=ax6)
    fig.suptitle(f'{name}\n beam intensity normalised single photons hits')
    fig.tight_layout(rect=[0.05, 0.05, .95, 0.95])
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
#     plt.show()
#     break
    plt.savefig(respath+'scanning_peaks.png')
    plt.close()

## more aggressive Filtering

In [ ]:
plt.rcParams['figure.figsize']=(15,12)
for filename in tqdm.tqdm(filenames):
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    
    inputfile=h5py.File(filename,'r')
    pulse=np.array(inputfile['pulse_energy_hutch_joule'])
    samplex=np.array(inputfile['motors/sampleX'])
    samplez=np.array(inputfile['motors/sampleZ'])+np.array(inputfile['motors/profZ'])

    pulse_ok=np.logical_and(pulse>0.00035,pulse<np.percentile(pulse,99.99))
    samplex_ok=np.logical_and(samplex<(np.max(samplex)-0.001),samplex>np.min(samplex)+0.001)
    shot_ok=np.logical_and(samplex_ok,pulse_ok)

    for detname in ['dual','octal']:

        intensity=np.array(inputfile[f'detectors/{detname}/intensity'])
        normintensity=intensity/pulse
        normintensity_f=normintensity[shot_ok]
        intensity_ok=np.abs(normintensity-np.mean(normintensity_f))<3*np.std(normintensity_f)
        detshotok=np.logical_and.reduce((intensity_ok,shot_ok))

        hist=np.array(inputfile[f'detectors/{detname}/hist/shot'])[detshotok,:]
        p8=np.sum(hist[:,1000:1100],axis=1)
        p6=np.sum(hist[:,750:850],axis=1)
        p11=np.sum(hist[:,1250:1350],axis=1)
        
        fig,((ax1,ax2),(ax3,ax4))=plt.subplots(nrows=2,ncols=2)
        
        im1=ax1.scatter(samplex[detshotok],samplez[detshotok],c=intensity[detshotok],s=2)
        ax1.set_title(f'intensity {detname} after filtering')
        plt.colorbar(im1,ax=ax1)

        ax2.hist(p11)
        ax2.set_title(f'scatter single photons {detname}')

        ax3.hist(p8)
        ax3.set_title(f'copper single photons {detname}')

        ax4.hist(p6)
        ax4.set_title(f'iron single photons {detname}')
        plt.suptitle(f'{name} {detname}')
        fig.tight_layout(rect=[0.05, 0.05, .95, 0.95])
        respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
        plt.savefig(respath+f'{detname}_singlephotons_hist.png')
        plt.close()

## masking

In [ ]:
##needs p2 as input
plt.rcParams['figure.figsize']=(20,20)
o=h5py.File('/home/gorkhover/work/testp29.h5','r')

fig,((ax1,ax2),(ax3,ax4))=plt.subplots(nrows=2,ncols=2)

photon_evmean=np.array(o['dual/photons/mean'])
photon_evmax=np.array(o['dual/photons/mean'])

photon_evstd=np.array(o['dual/photons/std'])

photon_n=np.array(o['dual/photons/n'])
photon_evmean[photon_n==0]=np.nan

mask_meandeviation=np.nan_to_num((np.abs((photon_evmean-np.nanmean(photon_evmean))/np.nanstd(photon_evmean)))<6)
mask_meandeviation=~skm.binary_dilation(~mask_meandeviation,np.ones((9,9)))
mask_border=~skm.binary_dilation(skm.binary_opening(photon_n==0,np.ones((5,5))),np.ones((30,30)))
mask=np.logical_and(mask_border,mask_meandeviation)

im1=ax1.matshow(photon_evmean,vmin=7000,vmax=9000)
plt.colorbar(im1,ax=ax1)
ax1.set_title('mean ev of single hits')

im2=ax2.matshow(photon_n)
plt.colorbar(im2,ax=ax2)
ax2.set_title('count of single hits')

im3=ax3.matshow(photon_evstd)
plt.colorbar(im3,ax=ax3)
ax3.set_title('std ev of single hits')

ax4.matshow(mask)
ax4.set_title('mask')
plt.savefig('photons.png',dpi=400)
plt.show()

print(f'will keep {np.sum(mask)} of {np.sum(photon_n!=0)} pixels ({np.round(np.sum(mask)/np.sum(photon_n!=0)*100,1)}%)')
print(f'will keep {np.sum(photon_n[mask])} of {np.sum(photon_n)} single photons ({np.round(np.sum(photon_n[mask])/np.sum(photon_n)*100,2)}%)')


In [ ]:
plt.rcParams['figure.figsize']=(20,20)

outfile=h5py.File('/home/gorkhover/work/TAIS2020/foil_mask.h5','w')
for detname in ('dual','octal'):
    maskhigh=None 
    masklow=None
    maskborder=None
    for i,filename in enumerate(tqdm.tqdm(filenames)):
        name=os.path.splitext(os.path.basename(filename))[0][:-3]
        inputfile=h5py.File(filename,'r')
        firstrun=int(inputfile['meta/runs'][0])
        lastrun=int(inputfile['meta/runs'][-1])
        bg='dark_foil2' if firstrun>=918639 else 'dark_foil'
        s=np.array(inputfile[f'detectors/{detname}/std'])
        m=np.array(inputfile[f'detectors/{detname}/mean'])
        if maskhigh is None or masklow is None or maskborder is None:
            maskhigh=np.zeros(m.shape,bool) 
            masklow=np.zeros(m.shape,bool) 
            maskborder=np.zeros(m.shape,bool) 
        cborder=skm.binary_opening(m<5,np.ones((5,5)))
        cborder=skm.binary_dilation(cborder,np.ones((41,41)))
        mm=np.copy(m)
        sm=np.copy(s)
        sm[cborder]=np.nan
        mm[cborder]=np.nan
        ts=6
        tm=6
        cmasks=np.logical_or(s<(np.nanmean(sm)-ts*np.nanstd(sm)), s>np.nanmean(sm)+ts*np.nanstd(sm))
        cmaskm=np.logical_or(m<(np.nanmean(mm)-tm*np.nanstd(mm)), m>np.nanmean(mm)+tm*np.nanstd(mm))
        maskhigh[cmasks]=1
        maskhigh[cmaskm]=1
        ts=3
        tm=3
        cmasks=np.logical_or(s<(np.nanmean(sm)-ts*np.nanstd(sm)), s>np.nanmean(sm)+ts*np.nanstd(sm))
        cmaskm=np.logical_or(m<(np.nanmean(mm)-tm*np.nanstd(mm)), m>np.nanmean(mm)+tm*np.nanstd(mm))
        masklow[np.logical_or(skm.binary_opening(cmaskm,np.ones((3,1))),skm.binary_opening(cmaskm,np.ones((1,3))))]=1
        masklow[np.logical_or(skm.binary_opening(cmasks,np.ones((3,1))),skm.binary_opening(cmasks,np.ones((1,3))))]=1
        maskborder[cborder]=1
    mask=np.logical_or.reduce((maskhigh,masklow,maskborder))
    mask=skm.binary_dilation(mask,np.ones((3,3)))
    mask=skm.binary_closing(mask,np.ones((3,3)))
    mask=~mask
    plt.matshow(mask)
    plt.title(f'background foils {detname}')
    plt.savefig(f'mask_{detname}_foil.png',dpi=400)
    plt.show()
    outfile[f'{detname}']=mask
outfile.close()
#     plt.matshow(masks,vmin=0,vmax=1)
#     plt.title(f'{name} ({i}/{firstrun}-{lastrun}/{bg}): std')
#     plt.colorbar()
#     #plt.savefig(f'std_{i}.png',dpi=400)
#     plt.show()
    
    
#     plt.matshow(maskm,vmin=0,vmax=1)
#     #plt.matshow(m,vmin=np.nanmean(m)-tm*np.nanstd(m),vmax=np.nanmean(m)+tm*np.nanstd(m))
#     plt.title(f'{name} ({i}/{firstrun}-{lastrun}/{bg}): mean')
#     plt.colorbar()
#     #plt.savefig(f'mean_{i}.png',dpi=400)
#     plt.show()
#maskold=np.copy(mask)


In [ ]:
##needs p2 as input

plt.rcParams['figure.figsize']=(20,20)
mask=np.array(h5py.File('/home/gorkhover/work/TAIS2020/foil_mask.h5')['dual']).astype(bool)

o=h5py.File('/home/gorkhover/work/testp29.h5','r')

fig,((ax1,ax2),(ax3,ax4))=plt.subplots(nrows=2,ncols=2)

photon_evmean=np.array(o['dual/photons/mean']).astype(float)
photon_evstd=np.array(o['dual/photons/std']).astype(float)
photon_n=np.array(o['dual/photons/n']).astype(float)

photon_evmean[~mask]=np.nan
photon_n[~mask]=np.nan
photon_evstd[~mask]=np.nan

im1=ax1.matshow(photon_evmean,vmin=7500,vmax=8500)
plt.colorbar(im1,ax=ax1)
ax1.set_title('mean ev of single hits')

im2=ax2.matshow(photon_n)
plt.colorbar(im2,ax=ax2)
ax2.set_title('count of single hits')

im3=ax3.matshow(photon_evstd)
plt.colorbar(im3,ax=ax3)
ax3.set_title('std ev of single hits')

ax4.matshow(mask)
ax4.set_title('mask used')
plt.savefig('photons_masked.png',dpi=800)
plt.show()

print(f'will keep {np.sum(mask)} of {np.sum(photon_n!=0)} pixels ({np.round(np.sum(mask)/np.sum(photon_n!=0)*100,1)}%)')
print(f'will keep {np.sum(photon_n[mask])} of {np.sum(photon_n)} single photons ({np.round(np.sum(photon_n[mask])/np.sum(photon_n)*100,2)}%)')


## beam spectrum

In [ ]:
beamspectrum=inputfile['beamspectrum'][0:2000,400:550]

In [ ]:
filtered=ss.savgol_filter(beamspectrum,21,3)
rms=np.sqrt(np.mean(np.square(beamspectrum-filtered),axis=1))

In [ ]:
plt.rcParams['figure.figsize']=(10,5)
plt.plot(beamspectrum.T)
plt.show()
plt.plot(beamspectrum[:5,:].T,'*-')
plt.show()
plt.plot(filtered[:5,:].T)
plt.show()
plt.plot((beamspectrum-filtered)[:5,:].T)
plt.show()

In [ ]:
plt.rcParams['figure.figsize']=(15,12)
for filename in tqdm.tqdm(filenames):
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    
    inputfile=h5py.File(filename,'r')
    pulse=np.array(inputfile['pulse_energy_hutch_joule'])
    ev=np.array(inputfile['photonEnergy'])
    p=5
    fig,((ax1,ax2),(ax3,ax4),(ax5,ax6),(ax7,ax8))=plt.subplots(nrows=4,ncols=2)
    pulseishigh=pulse>np.percentile(pulse,100-p)
    pulseislow=pulse<np.percentile(pulse,p)
    
    bs_low=np.array(inputfile['beamspectrum'][pulseislow,340:600])
    ids=np.argsort(pulse[pulseislow])
    bs_low=bs_low[ids,:]
    pulselow=pulse[pulseislow][ids]
    ev_low=ev[pulseislow][ids]
    
    bs_high=np.array(inputfile['beamspectrum'][pulseishigh,340:600])
    ids=np.argsort(pulse[pulseishigh])
    bs_high=bs_high[ids,:]
    pulsehigh=pulse[pulseishigh][ids]
    ev_high=ev[pulseishigh][ids]

# pulsebeam=inputfile['pulse_energy_beam_joule'][:2000]
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    for bs,ev,pulse,(axw,axbs,axdist,axmean,axrms,axev),label in zip((bs_low,bs_high),(ev_low,ev_high),(pulselow,pulsehigh),[(ax1,ax3,ax5,ax6,ax7,ax8),(ax2,ax4,ax5,ax6,ax7,ax8)],('low','high')):
        axw.matshow(bs.T,aspect="auto")
        axw.set_title(f'{label} beamspectrum over shots')
        axw.set_ylim(80,200)
        axbs.plot(bs[::bs.shape[0]//10,:].T/1000,'*-')
        axbs.set_ylim(0,140)
        axbs.set_xlim(80,200)
        axbs.set_title(f'{label} beamspectrum')
        
        filtered=ss.savgol_filter(bs,21,3)
        rms=np.sqrt(np.nanmean(np.square(bs-filtered),axis=1))
        axrms.hist(rms,bins=50,range=(0,15000),label=label,alpha=0.7)
        axrms.legend()
        axrms.set_title('rms')
        
        cg=np.nansum(bs*np.arange(0,bs.shape[1]),axis=1)/np.nansum(bs,axis=1)
        axmean.hist(cg,bins=50,range=(80,200),label=label,alpha=0.7)
        axmean.legend()
        axmean.set_title('center')
        
        dist=np.nansum(bs*np.abs(np.arange(0,bs.shape[1])[None,:]-cg[:,None]),axis=1)/np.nansum(bs,axis=1)
        axdist.hist(dist,bins=50,range=(10,40),label=label,alpha=0.7)
        axdist.legend()
        axdist.set_title('width')
        
        ev[np.isnan(ev)]=10920
        axev.text(10915,50,'nan')
        axev.hist(ev,bins=20,range=(10920,11040),label=label,alpha=0.7)
        axev.legend()
        axev.set_title('reported photon energy')
        axev.set_xlim(11050,10900)
    fig.suptitle(name)
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
    plt.savefig(respath+'beamspectrum.png')
    plt.close()

## photon count ##

In [ ]:
from accum import *


N=500
peaks=[
    ['Cu',8000],
    ['Fe',6400]
]
accums={}
for detname in ['dual']:
    for filename in tqdm.tqdm(filenames):
        name=os.path.splitext(os.path.basename(filename))[0][:-3]
        accums[name]=collections.defaultdict(accumulator)
        inputfile=h5py.File(filename,'r')
        det=inputfile[f'detectors/{detname}/data']
        ranges= [(f'{p[0]}_{n}',n*p[1]-1500,n*p[1]+1500) for p,n in itertools.product(peaks,(1,2,3))]
        ranges.extend([(f'stray_{n}',n*np.round(np.nanmean(inputfile['photonEnergy']))-1000,n*np.round(np.nanmean(inputfile['photonEnergy']))+1000) for n in (1,2,3)])

        skip=int((det.shape[0]-1)/N)
        for idet in tqdm.tqdm(range(0,N*skip,skip)):
            if idet>=det.shape[0]-1:
                break
            im=np.array(det[idet,...])
            for r in ranges:
                mask=np.logical_and(im>r[1],im<r[2]).astype(np.float32)
                accums[name][r[0]].add(im.astype(np.float32),mask)
       
        fig,ax=plt.subplots(6,3,figsize=(15,30))
        fig.tight_layout(rect=[0, 0, 1, 0.95])
        fig.suptitle(f' {name} {detname}')

        for i,(k,v) in enumerate(accums[name].items()): 
            cax=ax.flatten()[i]
            im=cax.matshow(v.mean,vmin=ranges[i][1],vmax=ranges[i][2])
            plt.colorbar(im,ax=cax)
            cax.set_title(f'{k} mean')

        for i,(k,v) in enumerate(accums[name].items()): 
            cax=ax.flatten()[i+9]
            im=cax.matshow(v.n*(1000./N),vmin=0)
            plt.colorbar(im,ax=cax)
            cax.set_title(f'{k} n / 1000 shots')
        respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
        plt.savefig(respath+f'photons_{detname}.png')
        plt.close()

## intensity thresholds ##

In [ ]:
for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    fig,axs=plt.subplots(nrows=2,figsize=(15,20))
    p1file=h5py.File(filename,'r')
    pulsebeam=np.array(p1file['pulse_energy_beam_joule'])
    pulsehutch=np.array(p1file['pulse_energy_beam_joule'])

    photonEnergy=np.array(p1file['photonEnergy'])
    samplex=np.array(p1file['motors']['sampleX'])
    pulse_ok = np.logical_and(pulsebeam > 450 * 1e-6, pulsebeam < (np.mean(pulsebeam) + 3 * np.std(pulsebeam)))
    samplex_ok = np.logical_and(samplex < (np.max(samplex) - 0.001), samplex > np.min(samplex) + 0.001)
    energy_ok = np.abs(np.nan_to_num(photonEnergy) - np.nanmedian(photonEnergy)) < (2 * np.nanstd(photonEnergy))

    shot_ok = np.logical_and.reduce((samplex_ok, pulse_ok,energy_ok))
    
    for detname,ax in zip(('dual','octal'),axs):
        intensity=np.array(p1file[f'detectors/{detname}/intensity'])

        intensity_f = intensity[shot_ok]/pulsehutch[shot_ok]    
        ax.plot(np.array(intensity_f),'.',markersize=2)

        intthresl=np.clip((np.mean(intensity_f)-1*np.std(intensity_f)),np.nanpercentile(intensity_f,.1),np.nanpercentile(intensity_f,5))
        intthresh=np.clip((np.mean(intensity_f)+3*np.std(intensity_f)),np.nanpercentile(intensity_f,99.9),np.inf)
    #     print(np.nanpercentile(intensity_f,.1),np.nanpercentile(intensity_f,5),(np.mean(intensity_f)-1*np.std(intensity_f)),intthresl)
    #     print(np.nanpercentile(intensity_f,99.9),(np.mean(intensity_f)+4*np.std(intensity_f)),intthresh)
    #     print(np.sum(intensity_f<intthresl),np.sum(intensity_f>intthresh))
        ax.axhline(intthresh,color='red')
        ax.axhline(intthresl,color='green')
        ax.set_title(detname)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle(f'{os.path.basename(filename)[:-6]}\n normalised intensity and thresholds')
    respath=f'/home/gorkhover/UserDatagorkhover/TAIS2020/foils/res/{name}/'
    plt.savefig(respath+'intensitythreshold.png')
    plt.close()
    

In [ ]:
def cutoctal(data):
    return np.concatenate((
        data[125:1225,80:1180],
        data[1225:-74,130:1230],
        data[75:1175,1175:-124],
        data[1175:-124,1230:-69]
    ),axis=0)